In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.mo_gaal import MO_GAAL
from pyod.models.so_gaal import SO_GAAL

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
import pickle

In [2]:
from imblearn.datasets import make_imbalance

with open('train.pk', 'rb') as f:
    xx, yy = pickle.load(f)
with open('test.pk', 'rb') as f:
    X_test,y_test = pickle.load(f)
X,Y = make_imbalance(xx, yy, sampling_strategy={'normal':27000, 'injection':1000, 'impersonation':1000, 'flooding':1000},random_state=0)
#xx, yy = make_imbalance(X, Y, sampling_strategy={'normal':3000, 'injection':1000, 'impersonation':1000, 'flooding':1000},random_state=0)

In [3]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

pd.options.display.max_rows
pd.set_option('display.max_rows', None)

In [4]:
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA', 'AutoEncoder', 'VAE', 'MO_GAAL', 'SO_GAAL']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

In [5]:
outliers_fraction = 0.1
outliers_percentage = 10

X_train_norm, X_test_norm = X, X_test

In [6]:
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

encoder = LabelEncoder()
encoded_y = encoder.fit_transform(Y)

binarizer = LabelBinarizer()
binarized_y = binarizer.fit_transform(encoded_y)
binarized_y.shape

encoded_y_test = encoder.transform(y_test)

y = keras.utils.to_categorical(encoded_y)
y_test = keras.utils.to_categorical(encoded_y_test)
print(encoded_y)

[0 0 0 ... 3 3 3]


In [7]:
np.count_nonzero(encoded_y == 3)

27000

In [8]:
for i in range(0,encoded_y.shape[0]):
    if encoded_y[i]==0:
        encoded_y[i]=1
    if encoded_y[i]==2:
        encoded_y[i]=1
for i in range(0,encoded_y.shape[0]):
    if encoded_y[i]==3:
        encoded_y[i]=0
print(np.count_nonzero(encoded_y == 0))
print(np.count_nonzero(encoded_y == 1))#outlier

for i in range(0,encoded_y_test.shape[0]):
    if encoded_y_test[i]==0:
        encoded_y_test[i]=1
    if encoded_y_test[i]==2:
        encoded_y_test[i]=1
for i in range(0,encoded_y_test.shape[0]):
    if encoded_y_test[i]==3:
        encoded_y_test[i]=0
print(np.count_nonzero(encoded_y_test == 0))
print(np.count_nonzero(encoded_y_test == 1))#outlier

27000
3000
530785
44858


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [10]:
    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state)
    }
    for clf_name, clf in classifiers.items():
        clf.fit(X_train_norm)
        
        #test_scores = clf.decision_function(X_train_norm)
        #roc = round(roc_auc_score(encoded_y, test_scores), ndigits=4)
        #prn = round(precision_n_scores(encoded_y, test_scores), ndigits=4)
        #print(clf_name)
        #print(clf_name,"Train dataset:")
        #print('ROC:{roc}, precision @ rank n:{prn}'.format(roc=roc, prn=prn))
        
        
        #test_scores1 = clf.decision_function(X_test_norm)
        #roc1 = round(roc_auc_score(encoded_y_test, test_scores1), ndigits=4)
        #prn1 = round(precision_n_scores(encoded_y_test, test_scores1), ndigits=4)
        #print("Test dataset:")
        #print('ROC:{roc1}, precision @ rank n:{prn1}'.format(roc1=roc1, prn1=prn1))
        
        predictions = clf.predict(X_train_norm)
        print(clf_name)
        print("Train dataset accuracy:");
        acc= accuracy_score(encoded_y, predictions)
        pre= precision_score(encoded_y, predictions)
        re= recall_score(encoded_y, predictions, average=None)
        print('Accuracy:{acc}, Precision:{pre}, Recall:{re}'.format(acc=acc, pre=pre, re=re))
        
        predictions1 = clf.predict(X_test_norm)
        print("Test dataset accuracy:");
        acc1= accuracy_score(encoded_y_test, predictions1)
        pre1= precision_score(encoded_y_test, predictions1)
        re1= recall_score(encoded_y_test, predictions1, average=None)
        print('Accuracy:{acc1}, Precision:{pre1}, Recall:{re1}\n\n'.format(acc1=acc1, pre1=pre1, re1=re1))
        

Angle-based Outlier Detector (ABOD)
Train dataset accuracy:
Accuracy:0.8093333333333333, Precision:0.03488372093023256, Recall:[0.89548148 0.034     ]
Test dataset accuracy:
Accuracy:0.07837843941470668, Precision:0.07796198358310899, Recall:[4.89840519e-04 1.00000000e+00]


Cluster-based Local Outlier Factor
Train dataset accuracy:
Accuracy:0.8910666666666667, Precision:0.4553333333333333, Recall:[0.93948148 0.45533333]
Test dataset accuracy:
Accuracy:0.8650587256337695, Precision:0.2597505270555165, Recall:[0.90474109 0.39551474]


Feature Bagging
Train dataset accuracy:
Accuracy:0.8276666666666667, Precision:0.10886806056236481, Recall:[0.90844444 0.10066667]
Test dataset accuracy:
Accuracy:0.08143067838920998, Precision:0.07814047396538017, Recall:[0.00387728 0.999086  ]


Histogram-base Outlier Detection (HBOS)
Train dataset accuracy:
Accuracy:0.8037333333333333, Precision:0.018666666666666668, Recall:[0.89096296 0.01866667]
Test dataset accuracy:
Accuracy:0.8208750909852113, Prec

In [16]:
classifiers1 = {'Auto Encoder': AutoEncoder(contamination=outliers_fraction, random_state=random_state, hidden_neurons= [16,8,8,16]),
         'VAE': VAE(contamination=outliers_fraction, random_state=random_state, encoder_neurons= [16,8,16], decoder_neurons= [16,8,16]),
        'MO_GAAL': MO_GAAL(contamination=outliers_fraction),
        'SO_GAAL': SO_GAAL(contamination=outliers_fraction)
               }
for clf_name, clf in classifiers1.items():
        clf.fit(X_train_norm)
        
        #test_scores = clf.decision_function(X_train_norm)
        #roc = round(roc_auc_score(encoded_y, test_scores), ndigits=4)
        #prn = round(precision_n_scores(encoded_y, test_scores), ndigits=4)
        #print(clf_name)
        #print(clf_name,"Train dataset:")
        #print('ROC:{roc}, precision @ rank n:{prn}'.format(roc=roc, prn=prn))
        
        
        #test_scores1 = clf.decision_function(X_test_norm)
        #roc1 = round(roc_auc_score(encoded_y_test, test_scores1), ndigits=4)
        #prn1 = round(precision_n_scores(encoded_y_test, test_scores1), ndigits=4)
        #print("Test dataset:")
        #print('ROC:{roc1}, precision @ rank n:{prn1}'.format(roc1=roc1, prn1=prn1))
        
        predictions = clf.predict(X_train_norm)
        print(clf_name)
        print("Train dataset accuracy:");
        acc= accuracy_score(encoded_y, predictions)
        pre= precision_score(encoded_y, predictions)
        re= recall_score(encoded_y, predictions, average=None)
        print('Accuracy:{acc}, Precision:{pre}, Recall:{re}'.format(acc=acc, pre=pre, re=re))
        
        predictions1 = clf.predict(X_test_norm)
        print("Test dataset accuracy:");
        acc1= accuracy_score(encoded_y_test, predictions1)
        pre1= precision_score(encoded_y_test, predictions1)
        re1= recall_score(encoded_y_test, predictions1, average=None)
        print('Accuracy:{acc1}, Precision:{pre1}, Recall:{re1}\n\n'.format(acc1=acc1, pre1=pre1, re1=re1))
        

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 153)               23562     
_________________________________________________________________
dropout (Dropout)            (None, 153)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 153)               23562     
_________________________________________________________________
dropout_1 (Dropout)          (None, 153)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 1

844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 0.6906
Epoch 62/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5789 - val_loss: 0.6906
Epoch 63/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 0.6906
Epoch 64/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 0.6906
Epoch 65/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 0.6906
Epoch 66/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5789 - val_loss: 0.6906
Epoch 67/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5789 - val_loss: 0.6906
Epoch 68/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 0.6906
Epoch 69/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 0.6906
Epoch 70/100
844/844 [==============================] - 2s 2ms/step - loss: 0.5790 - val_loss: 

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 153)]        0                                            
__________________________________________________________________________________________________
functional_1 (Functional)       [(None, 2), (None, 2 26374       input_1[0][0]                    
__________________________________________________________________________________________________
functional_3 (Functional)       (None, 153)          2935        functional_1[0][2]               
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 153)          23562       input_1[0][0]                    
_______________________________________________________________________________________

844/844 [==============================] - 2s 2ms/step - loss: 92.9928 - val_loss: 63.2814
Epoch 19/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9898 - val_loss: 63.2814
Epoch 20/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9872 - val_loss: 63.2814
Epoch 21/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9812 - val_loss: 63.2814
Epoch 22/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9848 - val_loss: 63.2814
Epoch 23/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9867 - val_loss: 63.2814
Epoch 24/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9848 - val_loss: 63.2814
Epoch 25/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9815 - val_loss: 63.2814
Epoch 26/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9876 - val_loss: 63.2814
Epoch 27/100
844/844 [==============================] - 2s 2ms/step - loss: 9

844/844 [==============================] - 2s 3ms/step - loss: 93.0278 - val_loss: 63.2814
Epoch 97/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9837 - val_loss: 63.2814
Epoch 98/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9854 - val_loss: 63.2814
Epoch 99/100
844/844 [==============================] - 2s 2ms/step - loss: 92.9855 - val_loss: 63.2813
Epoch 100/100
844/844 [==============================] - 2s 2ms/step - loss: 93.0453 - val_loss: 63.2813
VAE
Train dataset accuracy:
Accuracy:0.8034666666666667, Precision:0.017333333333333333, Recall:[0.89081481 0.01733333]
Test dataset accuracy:
Accuracy:0.21624861242124024, Precision:0.0904360518447882, Recall:[0.15001178 1.        ]


Epoch 1 of 60

Testing for epoch 1 index 1:


...

16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 54 index 33:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 54 index 34:
16/16 [==============================] - 0s 2ms/step - loss: nan

Testing for epoch 54 index 35:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 54 index 36:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 54 index 37:
16/16 [==============================] - 0s 961us/step - loss: nan

Testing for epoch 54 index 38:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 54 index 39:
16/16 [==============================] - 0s 997us/step - loss: nan

Testing for epoch 54 index 40:
16/16 [==============================] - 0s 979us/step - loss: nan

Testing for epoch 54 index 41:
16/16 [==============================] - 0s 970us/step - loss: nan

Testing for epoch 54 index 42:
16/16 [===============

16/16 [==============================] - 0s 941us/step - loss: nan

Testing for epoch 55 index 57:
16/16 [==============================] - 0s 984us/step - loss: nan

Testing for epoch 55 index 58:
16/16 [==============================] - 0s 941us/step - loss: nan

Testing for epoch 55 index 59:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 55 index 60:
16/16 [==============================] - 0s 925us/step - loss: nan
Epoch 56 of 60

Testing for epoch 56 index 1:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 56 index 2:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 56 index 3:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 56 index 4:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 56 index 5:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 56 index 6:
16/16 [======

16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 57 index 21:
16/16 [==============================] - 0s 973us/step - loss: nan

Testing for epoch 57 index 22:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 57 index 23:
16/16 [==============================] - 0s 2ms/step - loss: nan

Testing for epoch 57 index 24:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 57 index 25:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 57 index 26:
16/16 [==============================] - 0s 976us/step - loss: nan

Testing for epoch 57 index 27:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 57 index 28:
16/16 [==============================] - 0s 2ms/step - loss: nan

Testing for epoch 57 index 29:
16/16 [==============================] - 0s 940us/step - loss: nan

Testing for epoch 57 index 30:
16/16 [=================

16/16 [==============================] - 0s 957us/step - loss: nan

Testing for epoch 58 index 42:
16/16 [==============================] - 0s 994us/step - loss: nan

Testing for epoch 58 index 43:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 58 index 44:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 58 index 45:
16/16 [==============================] - 0s 966us/step - loss: nan

Testing for epoch 58 index 46:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 58 index 47:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 58 index 48:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 58 index 49:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 58 index 50:
16/16 [==============================] - 0s 2ms/step - loss: nan

Testing for epoch 58 index 51:
16/16 [=================

16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 6:
16/16 [==============================] - 0s 991us/step - loss: nan

Testing for epoch 60 index 7:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 8:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 9:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 10:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 11:
16/16 [==============================] - 0s 2ms/step - loss: nan

Testing for epoch 60 index 12:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 13:
16/16 [==============================] - 0s 950us/step - loss: nan

Testing for epoch 60 index 14:
16/16 [==============================] - 0s 1ms/step - loss: nan

Testing for epoch 60 index 15:
16/16 [=======================